In [25]:
import sys
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Spotify API credentials
client_id = 'd7ecba8b249c437fa94175cbe74c5cad'
client_secret = '3ebaecf089cf4e81af1bd8172ee05b85'

# Initialize Spotipy with client credentials
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [26]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [27]:
hot_songs = pd.read_csv('hot_songs.csv')
not_hot_songs = pd.read_csv('not_hot_songs.csv')

In [28]:
hot_songs.head()

,Title,Artists
0,Rockin' Around The Christmas Tree,Brenda Lee
1,All I Want For Christmas Is You,Mariah Carey
2,Jingle Bell Rock,Bobby Helms
3,Last Christmas,Wham!
4,A Holly Jolly Christmas,Burl Ives


In [29]:
not_hot_songs = not_hot_songs.drop(['level_0'], axis=1)
not_hot_songs = not_hot_songs.drop(['index'], axis=1)
not_hot_songs.head()

,Artists,Title
0,Sam Feldt,Post Malone
1,Quality Control,100 Racks (Offset feat. Playboi Carti)
2,Drake,God's Plan
3,Francis and the Lights,May I Have This Dance (Remix)
4,Future,Trapped In The Sun


In [30]:
import pandas as pd
import time
import numpy as np

In [31]:
def search_song(title, artist, limit):
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                           client_secret=client_secret))
    try:
        results = sp.search(q=f"{title} {artist}", type='track', limit=limit)
        song_id = results['tracks']['items'][0]['id']
        return song_id
    except IndexError:
        print("Song not found!")
        return ""

def get_audio_features(list_of_song_ids):

    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                           client_secret=client_secret))

    audio_features_dict = {'id': [], 'danceability': [], 'energy': [],
                           'key': [], 'loudness': [], 'mode': [],
                           'speechiness': [], 'acousticness': [],
                           'instrumentalness': [], 'liveness': [],
                           'valence': [], 'tempo': []}
  
    chunks = np.array_split(list_of_song_ids, len(list_of_song_ids) // 50 + 1)

    for chunk in chunks:
        print("Collecting IDs for chunk...")
        time.sleep(20) 
        features = sp.audio_features(chunk)
        for feature in features:
            for key, value in feature.items():
                if key in audio_features_dict:
                    audio_features_dict[key].append(value)

    audio_features_df = pd.DataFrame(audio_features_dict)
    return audio_features_df

def add_audio_features(df, audio_features_df):
  
    merged_df = pd.merge(df, audio_features_df, on='id', how='inner')
    return merged_df

In [ ]:
#from functions import search_song, get_audio_features, add_audio_features

list_of_ids_hot = []
list_of_ids_not_hot = []

for index, row in hot_songs.iterrows():
    try:
        song_id = search_song(row['Title'], row['Artists'], limit=1)
        list_of_ids_hot.append(song_id)
    except:
        print("Error processing song:", row['Title'], row['Artists'])
        list_of_ids_hot.append("")

hot_songs['id'] = list_of_ids_hot

list_of_ids_hot = hot_songs['id'].tolist()
audio_features_hot = get_audio_features(list_of_ids_hot)

hot_songs_feat = add_audio_features(hot_songs, audio_features_hot)

In [ ]:
audio_features_hot

In [ ]:
hot_songs_feat

In [ ]:
for index, row in not_hot_songs.iterrows():
    try:
        song_id = search_song(row['Title'], row['Artists'], limit=1)
        list_of_ids_not_hot.append(song_id)
    except:
        print("Error processing song:", row['Title'], row['Artists'])
        list_of_ids_not_hot.append("")

not_hot_songs['id'] = list_of_ids_not_hot

list_of_ids_not_hot = not_hot_songs['id'].tolist()
audio_features_not_hot = get_audio_features(list_of_ids_not_hot)

not_hot_songs_feat = add_audio_features(not_hot_songs, audio_features_not_hot)



In [ ]:
not_hot_songs_feat

In [ ]:
not_hot_songs_feat.isna().sum()

In [ ]:
not_hot_songs_feat = not_hot_songs_feat.drop_duplicates(subset=['id'])

In [ ]:
not_hot_songs_feat.to_csv('not_hot_songs_with_audio.csv', index=False)

In [ ]:
hot_songs_feat.to_csv('hot_songs_with_audio.csv', index=False)